In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
# data collection
def load_data(data):
    # Load the train data from the train file
    # Read train CSV file and select 'Path' and 'Pneumonia' columns
    df = pd.read_csv(data, usecols=['Path', 'Pneumonia'])
    df = df[df['Pneumonia'].notna()]
    df = df.replace(-1, 0)
#     df = df[df['Pneumonia'] != -1]
    return df

In [3]:
train = load_data('CheXpert-v1.0-small/train.csv')
valid = load_data('CheXpert-v1.0-small/valid.csv')
train.shape, valid.shape

((27608, 2), (234, 2))

In [4]:
train['Pneumonia'].value_counts()

0.0    21569
1.0     6039
Name: Pneumonia, dtype: int64

In [5]:
batch_size = 32
image_size = (224, 224)
input_shape = (224, 224, 3)
num_views = 1
num_classes = 2
num_channels = 3

In [6]:
def load_image(path, image_size):
    img = cv2.imread(path)
    img = cv2.resize(img, image_size)
    return img

# Create augmented numpy array
def augment_data(data, labels):
    aug_data = []
    aug_labels = []
    for i in range(len(data)):
        # Add original data
        aug_data.append(data[i])
        aug_labels.append(labels[i])
        
        # Flip image horizontally
        flip = np.fliplr(data[i])
        aug_data.append(flip)
        aug_labels.append(labels[i])
        
        # Rotate image by 90 degrees
        rotate = np.rot90(data[i])
        aug_data.append(rotate)
        aug_labels.append(labels[i])
        
    aug_data = np.array(aug_data)
    aug_labels = np.array(aug_labels)
    return aug_data, aug_labels

In [7]:
# Load data
train_data = []
train_labels = []
for i, row in train.iterrows():
    path = row['Path']
    label = row['Pneumonia']
    img = load_image(path, image_size)
    train_data.append(img)
    train_labels.append(label)

valid_data = []
valid_labels = []
for i, row in valid.iterrows():
    path = row['Path']
    label = row['Pneumonia']
    img = load_image(path, image_size)
    valid_data.append(img)
    valid_labels.append(label)

# Augment data
train_data, train_labels = augment_data(train_data, train_labels)

In [8]:
# Make sure number of 0 and 1 labels are equal
num_zeros = len(train_labels[train_labels == 0])
num_ones = len(train_labels[train_labels == 1])
if num_zeros > num_ones:
    # Remove excess 0 labels and data
    remove_indices = np.random.choice(np.where(train_labels == 0)[0], num_zeros - num_ones, replace=False)
    train_data = np.delete(train_data, remove_indices, axis=0)
    train_labels = np.delete(train_labels, remove_indices)
elif num_ones > num_zeros:
    # Remove excess 1 labels and data
    remove_indices = np.random.choice(np.where(train_labels == 1)[0], num_ones - num_zeros, replace=False)
    train_data = np.delete(train_data, remove_indices, axis=0)
    train_labels = np.delete(train_labels, remove_indices)

In [9]:
# Shuffle data train data
train_indices = np.arange(len(train_labels))
np.random.shuffle(train_indices)
train_data = train_data[train_indices]
train_labels = train_labels[train_indices]

# valid data
valid_indices = np.arange(len(valid_labels))
valid_data_new = []
valid_labels_new = []
for i in valid_indices:
    valid_data_new.append(valid_data[i])
    valid_labels_new.append(valid_labels[i])

valid_data = valid_data_new
valid_labels = valid_labels_new

In [10]:
zero = 0
one = 0
for i in train_labels:
  if i == 0:
    zero += 1
  else:
    one += 1
zero, one

(18117, 18117)

In [11]:
train_data.shape

(36234, 224, 224, 3)

In [12]:
!pip install vit_keras
!pip install tensorflow_addons

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg
from tensorflow.keras.applications.vgg16 import VGG16
from vit_keras import vit
from keras.optimizers import Adam
from tensorflow.keras.models import Model

In [14]:
# define the model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in base_model.layers:
    layer.trainable = False

In [15]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [16]:
# compile the model
optimizer = Adam(lr=1e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\hira\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
history = model.fit(train_data, train_labels, 
                    epochs=10, validation_data=(valid_data, valid_labels), batch_size=32, verbose=1)

Epoch 1/10
 693/1133 [=================>............] - ETA: 2:42:47 - loss: 0.9514 - accuracy: 0.5113